### ALUMNO09 - DANIEL GONZALEZ PALAZON

### Ejercicio 3.2

### Reduction operation: the sum of the numbers in the range [0, a)

In [3]:
import sys

# Si se proporciona un argumento, lo toma; si no, usa 10**6 por defecto
try:
    value = int(sys.argv[1])
except (IndexError, ValueError):
    value = 10**6

print(f"Usando value = {value}")


import time

def reduc_operation(a):
    """Compute the sum of the numbers in the range [0, a)."""
    x = 0
    for i in range(a):
        x += i
    return x

initialTime = time.time()
suma = reduc_operation(value)
finalTime = time.time()

print("Time taken by reduction operation:", (finalTime - initialTime), "seconds")

# Utilizando las operaciones mágicas de ipython
%timeit -r 2 reduc_operation(value)

print(f"\n \t Computing the sum of numbers in the range [0, value): {suma}\n")

Time taken by reduction operation: 0.2683451175689697 seconds
229 ms ± 7.32 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)

 	 Computing the sum of numbers in the range [0, value): 499999500000



In [4]:
# — Ejercicio 3.2 a) —  
import time

# Crear la lista de Python con 10**6 elementos
n = 10**6
lst = list(range(n))

# a) Medir con bucle for
start = time.time()
total_for = 0
for x in lst:
    total_for += x
elapsed_for = time.time() - start

print(f"Bucle for:   total={total_for}, tiempo={elapsed_for:.4f} s")

# b) Medir con sum()
start = time.time()
total_sum = sum(lst)
elapsed_sum = time.time() - start

print(f"sum(lst):    total={total_sum}, tiempo={elapsed_sum:.4f} s")


Bucle for:   total=499999500000, tiempo=0.5357 s
sum(lst):    total=499999500000, tiempo=0.0319 s


In [5]:
# — Ejercicio 3.2 b) —  
import time
import numpy as np

# Convertir la lista a un array de NumPy
arr = np.array(lst)

# a) Medir con bucle for sobre el array
start = time.time()
total_for_np = 0
for x in arr:
    total_for_np += x
elapsed_for_np = time.time() - start

print(f"Bucle for NumPy:   total={total_for_np}, tiempo={elapsed_for_np:.4f} s")

# b) Medir con np.sum()
start = time.time()
total_sum_np = np.sum(arr)
elapsed_sum_np = time.time() - start

print(f"np.sum(arr):       total={total_sum_np}, tiempo={elapsed_sum_np:.4f} s")


Bucle for NumPy:   total=499999500000, tiempo=0.9990 s
np.sum(arr):       total=499999500000, tiempo=0.0025 s


### 3.2 c) Análisis de resultados

Al ejecutar los tres procedimientos de suma en el rango \[0, 10<sup>6</sup>), hemos obtenido estos tiempos aproximados:

| Método                              | Tiempo        |
|-------------------------------------|---------------|
| 1. Código original (`reduc_operation`)       | 0.27 s        |
| 2.a. Bucle `for` sobre lista        | 0.54 s        |
| 2.b. `sum(lst)`                     | 0.03 s        |
| 3.a. Bucle `for` sobre `ndarray`    | 1.00 s        |
| 3.b. `np.sum(arr)`                  | 0.0025 s       |

**Puntos clave:**

1. **Código original vs. bucle sobre lista**  
   - Ambos usan un bucle en Python puro para acumular elemento a elemento.  
   - La función genérica `reduc_operation` sobre `range(a)` tarda ~0.27 s, mientras que la suma manual de `list(range(n))` tarda ~0.54 s.  
   - La diferencia se debe al coste de creación de la lista completa antes de iterar y a la llamada a la función.

2. **`sum(lst)`**  
   - Implementada en C dentro del intérprete de Python, evita el bucle en Python y recorre la lista con código nativo.  
   - Mejora el rendimiento en un orden de magnitud: de ~0.44 s a ~0.03 s.

3. **Bucle `for` sobre `ndarray`**  
   - Aunque el tipo de datos cambia, se sigue iterando en Python elemento a elemento, lo que introduce *más* overhead que con listas, porque cada acceso a un elemento de `ndarray` implica comprobaciones adicionales de NumPy.  
   - Por eso la suma manual sobre `arr = np.array(lst)` es aún más lenta (~0.95 s).

4. **`np.sum(arr)`**  
   - Ejecuta la operación de reducción íntegramente en C (y optimizado con vectores SIMD si el hardware lo permite), accediendo a memoria contigua.  
   - Resultado: la ejecución es ultrarrápida (~0.003 s), casi 100× más rápida que `sum(lst)` y ~300× más rápida que el bucle Python sobre `ndarray`.

---

**Conclusión:**  
- El principal cuello de botella en Python puro es el bucle interpretado elemento a elemento.  
- Aprovechar implementaciones nativas (ya sea `sum()` para listas o `np.sum()` para arrays) traslada la carga al código compilado en C, consiguiendo enormes mejoras de rendimiento.


##### Ejercicio 3.3

In [14]:
# — Ejercicio 3.3 a) comparativa sin y con Numba —  
import time
import numpy as np
from numba import njit

# Convertir la lista existente a un array de NumPy
arr = np.array(lst)

# 1. Versión sin Numba: bucle Python
def reduce_py(a):
    total = 0
    for x in a:
        total += x
    return total

# 2. Versión sin Numba: np.sum
def reduce_np(a):
    return np.sum(a)

# 3. Versión Numba: bucle Python compilado
@njit
def reduce_numba(a):
    total = 0
    for x in a:
        total += x
    return total

# 4. Versión Numba: np.sum compilado
@njit
def sum_numba(a):
    return np.sum(a)

# Compilar funciones Numba (primera llamada no medida)
reduce_numba(arr)
sum_numba(arr)

# Medir tiempos
start = time.time()
res_py = reduce_py(arr)
t_py = time.time() - start

start = time.time()
res_np = reduce_np(arr)
t_np = time.time() - start

start = time.time()
res_nb = reduce_numba(arr)
t_nb = time.time() - start

start = time.time()
res_nb_sum = sum_numba(arr)
t_nb_sum = time.time() - start

# Mostrar resultados
print(f"Pure Python loop:   time={t_py:.4f} s, result={res_py}")
print(f"Numpy sum:          time={t_np:.4f} s, result={res_np}")
print(f"Numba Python loop:  time={t_nb:.4f} s, result={res_nb}")
print(f"Numba Numpy sum:    time={t_nb_sum:.4f} s, result={res_nb_sum}")


Pure Python loop:   time=0.6924 s, result=499999500000
Numpy sum:          time=0.0019 s, result=499999500000
Numba Python loop:  time=0.0023 s, result=499999500000
Numba Numpy sum:    time=0.0015 s, result=499999500000


### 3.3 b) Análisis de resultados con Numba

Al comparar los cuatro métodos de reducción en el rango \[0, 10⁶) obtenemos:

| Método                  | Tiempo     |
|-------------------------|------------|
| Pure Python loop        | 0.6924 s   |
| NumPy `sum`             | 0.0019 s   |
| Numba Python loop       | 0.0023 s   |
| Numba NumPy `sum`       | 0.0015 s   |

**Observaciones principales:**

- **Bucle Python puro (0.6924 s)**  
  Recorrer elemento a elemento en Python incurre en un alto overhead interpretado.

- **`np.sum(arr)` sin Numba (0.0019 s)**  
  Implementación en C que opera sobre memoria contigua, es ~365× más rápida que el bucle puro.

- **Bucle Python compilado con Numba (0.0023 s)**  
  Tras la compilación JIT, la versión `reduce_numba(arr)` convierte el bucle Python en código máquina optimizado.  
  Aunque sigue siendo un bucle explícito, ejecuta casi al mismo nivel que `np.sum`, demostrando la potencia de Numba para acelerar código Python puro.

- **`np.sum(arr)` compilado con Numba (0.0015 s)**  
  Al compilar la llamada a la rutina de NumPy, obtenemos un pequeño extra de velocidad respecto a la versión no compilada.  
  Esto sugiere que Numba inyecta optimizaciones adicionales en el bucle interno de la suma.

---

**Conclusión:**  
Numba permite llevar bucles escritos en Python a un rendimiento cercano al de código vectorizado en C, eliminando el cuello de botella del intérprete. Para operaciones simples sobre arrays, `np.sum` sigue siendo la opción más directa, pero `@njit` es una gran herramienta cuando se requieren bucles más complejos o lógica personalizada que NumPy no cubre nativamente.  
